In [1]:
"""  
After using LLMs to find similar books to each other and then classify them (Fiction/Nonfiction), time for sentiment analysis that is going to determine the emotional tone.
Target is classifying data into 7 categories:
anger, disgust, fear, joy, sadness, joy, surprise, and neutral
"""
import pandas as pd 
books = pd.read_csv("books_with_categories.csv")

In [2]:
"""
The reported evaluation accuracy for this model is 66%. This is considered significantly higher than a random-chance baseline, which for 7 emotion classes would be 1/7 or approximately 14%. 
"""


from transformers import pipeline
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k = None,
                      device= "cuda"
                      )
classifier("I love this!")

Device set to use cuda


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528688922524452},
  {'label': 'neutral', 'score': 0.0057645998895168304},
  {'label': 'anger', 'score': 0.004419785924255848},
  {'label': 'sadness', 'score': 0.002092393347993493},
  {'label': 'disgust', 'score': 0.0016119939973577857},
  {'label': 'fear', 'score': 0.0004138521908316761}]]

In [3]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [4]:
#Classify the whole description
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548401117324829},
  {'label': 'neutral', 'score': 0.16985271871089935},
  {'label': 'sadness', 'score': 0.11640920490026474},
  {'label': 'surprise', 'score': 0.02070067822933197},
  {'label': 'disgust', 'score': 0.019100727513432503},
  {'label': 'joy', 'score': 0.015161407180130482},
  {'label': 'anger', 'score': 0.003935148939490318}]]

In [5]:
#Classify each sentence inside the description
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296029329299927},
  {'label': 'neutral', 'score': 0.14038559794425964},
  {'label': 'fear', 'score': 0.06816216558218002},
  {'label': 'joy', 'score': 0.04794234409928322},
  {'label': 'anger', 'score': 0.009156346321105957},
  {'label': 'disgust', 'score': 0.0026284747291356325},
  {'label': 'sadness', 'score': 0.0021221598144620657}],
 [{'label': 'neutral', 'score': 0.44937023520469666},
  {'label': 'disgust', 'score': 0.27359238266944885},
  {'label': 'joy', 'score': 0.10908287763595581},
  {'label': 'sadness', 'score': 0.09362705796957016},
  {'label': 'anger', 'score': 0.040478311479091644},
  {'label': 'surprise', 'score': 0.026970146223902702},
  {'label': 'fear', 'score': 0.006879040505737066}],
 [{'label': 'neutral', 'score': 0.6462163925170898},
  {'label': 'sadness', 'score': 0.24273303151130676},
  {'label': 'disgust', 'score': 0.04342260584235191},
  {'label': 'surprise', 'score': 0.02830052748322487},
  {'label': 'joy', 'score': 0.01421

In [7]:
#The idea is for each book has a separate column of each emotion

from tqdm import tqdm
import numpy as np

def calculate_max_emotion_scores(predictions, emotion_labels):
    """
    Calculates the maximum score for each emotion across all sentences for a single book.
    Uses list comprehensions for concise score extraction.

    Args:
        predictions (list of list of dict): Output from the Hugging Face classifier
                                            for one book's sentences.
        emotion_labels (list): A list of all possible emotion labels.

    Returns:
        dict: A dictionary where keys are emotion labels and values are the
              maximum score observed for that emotion across all sentences.
    """
    # Create a flattened list of all individual emotion prediction dicts
    # from all sentences.
    all_single_predictions = [item for sublist in predictions for item in sublist]

    max_scores_per_emotion = {}
    for label in emotion_labels:
        # Extract all scores for the current 'label' across ALL sentences
        scores_for_this_label = [
            item['score'] for item in all_single_predictions if item['label'] == label
        ]
        # Find the maximum score for this label, or 0.0 if no scores were found
        max_scores_per_emotion[label] = np.max(scores_for_this_label) if scores_for_this_label else 0.0

    return max_scores_per_emotion

In [8]:
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]

# Initialize a list to store results for each book
# Each element in this list will be a dictionary containing ISBN and emotion scores for one book.
results_list = []

for index, row in tqdm(books.iterrows(), total=len(books), desc="Processing Books"):
    isbn = row["isbn13"]
    description = row["description"]

    sentences = description.split(".")
    predictions = classifier(sentences)
    
    # Pass emotion_labels to the calculation function
    max_scores = calculate_max_emotion_scores(predictions, emotion_labels)

    # Create a dictionary for the current book's results
    book_result = {"isbn13": isbn}
    book_result.update(max_scores) # Add all emotion scores to the dictionary

    results_list.append(book_result)


Processing Books: 100%|██████████| 5197/5197 [02:25<00:00, 35.69it/s]


In [10]:
emotions_df = pd.DataFrame(results_list)

In [11]:
emotions_df.head()

,isbn13,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0.064134,0.273592,0.928169,0.932798,0.967158,0.729603,0.646216
1,9780002261982,0.612619,0.348285,0.942528,0.704422,0.111690,0.252545,0.887940
2,9780006178736,0.064134,0.104007,0.972321,0.767237,0.111690,0.078766,0.549477
3,9780006280897,0.351483,0.150722,0.360707,0.251881,0.111690,0.078766,0.732685
4,9780006280934,0.081412,0.184496,0.095043,0.040564,0.475881,0.078766,0.884390


In [12]:
books = pd.merge(books, emotions_df, on = "isbn13", how="left")

In [13]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273592,0.928169,0.932798,0.967158,0.729603,0.646216
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612619,0.348285,0.942528,0.704422,0.111690,0.252545,0.887940
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767237,0.111690,0.078766,0.549477
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351483,0.150722,0.360707,0.251881,0.111690,0.078766,0.732685
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184496,0.095043,0.040564,0.475881,0.078766,0.884390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148208,0.030643,0.919165,0.255170,0.980877,0.030656,0.853722
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.111690,0.227765,0.883198
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339218,0.947779,0.066685,0.057625,0.375755
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459269,0.759456,0.368111,0.078766,0.951104


In [14]:
books.to_csv("books_with_emotions.csv", index = False)